In [170]:
####################
#set up packages 
####################
import ee
import geemap
import numpy as np
import ipyleaflet
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import geemap.colormaps as cm





##Program to test mapping##

In [171]:

####################################
#country list to loop through
####################################
Map=geemap.Map() #setting up the map to display 
countries = ee.FeatureCollection('FAO/GAUL/2015/level0') #countries is the dataset
 
countrylist=['Bahamas','Barbados','Cuba','Dominican Republic','Haiti','Trinidad and Tobago']#['Jamaica','Puerto Rico',
stormlist=['hellow','test']


In [172]:
#visualisation parameteres are set up
Palette= cm.palettes.brg

Vis_params= {
    'min': -100,
    'max': 100,
    'palette': Palette}


def threshold(img, level):
    mask=img.lt(level)
    #mask=mask.selfMask()
    return mask

Start_date='2017-08-01'
End_date='2017-10-01'

#seting up nightlight VIIRS monthly
Nightlight= ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG").select('avg_rad') #the dataset and band selected
JamaicanNight= Nightlight.filterDate(Start_date,'2021-01-01') #the dataset is cutdown to the relivant dates and region removed .filterbounds(roi)





In [173]:
t_level=5

def func(x,y):


    countryname=x
    roi = countries.filter(ee.Filter.eq('ADM0_NAME', countryname)) #roi is a subset of the countries dataset 
    roigeom=roi.geometry() #set up as a geometry to allow .filterBounds and .clip would work in other ways but this is most efficient
    roicenter=roigeom.centroid() #this allows the map to be centered on the center of the roi
    Map.addLayer(roigeom,{},'Region of Interest')
    Map.centerObject(roi,5)

    stormname=y
    print(stormname)

    # print(type(roi))
    #before image is set up
    Before = ee.Image(JamaicanNight.filterDate(Start_date).mean())
    Map.addLayer(Before.clipToCollection(roi),{},'Before2')
    Before = Before.clipToCollection(roi)

    #before image is thresholded and mask set up
    Before_mask = threshold(Before, t_level)
    Before_mask = Before_mask.clipToCollection(roi)
    Before_masked=Before.where(Before_mask,0)

    #After image set up
    After = ee.Image(JamaicanNight.filterDate(End_date).mean())
    Map.addLayer(After.clipToCollection(roi),{},'After 2')
    After = After.clipToCollection(roi)

    #After image is thresholded and mask set up 
    # After_mask = threshold(After, t_level)
    # After_mask = After_mask.clip(roi)
    After_masked=After.where(Before_mask,0) #here before mask is used because if the light drops to less than threshold in interesting areas this is useful information

    #Before and after added to map
    Map.addLayer(Before_mask,{},'Before mask')
    Map.addLayer(Before_masked,{},'Before')
    Map.addLayer(After_masked,{},'After')


    #the calculation for the percentage difference set up (could be written as 1 equation)
    pd=After_masked.subtract(Before_masked)
    pd=pd.divide(Before_masked)
    pd=pd.multiply(100)

    #pd mask is set up, the .mask function every pixel with or lower value in the given image are transparent, which is why before.not() is used
    pd_mask = Before_mask.Not()
    pd_mask = pd_mask.clipToCollection(roi)
    pd_masked=pd.mask(pd_mask)

    #visualisation for export set up
    export=pd_masked.visualize(**{
        'min': -20,
        'max': 20,
        'palette': ['blue','red','green']}).clip(roi) #was manually entered for some reason the previous parameters where not acepted because of the palette

    #added to map with color bar
    Map.addLayer(pd_masked,Vis_params,'Difference')
    Map.add_colorbar(Vis_params, label='percentage difference in nightlight',layer_name='color bar')
    Map.addLayer(export,{},'export')
    
  

dropdown = interact(func, x=countrylist, y=stormlist)
Map


interactive(children=(Dropdown(description='x', options=('Bahamas', 'Barbados', 'Cuba', 'Dominican Republic', …

Map(center=[24.202056257216125, -76.48643545294412], controls=(WidgetControl(options=['position', 'transparent…

In [174]:
# def test1(x,y):
#     print(x)
#     print(y)



# test=interactive(test1,x=['banana','test'],y=['apples','grapes'])

# test
